In [258]:
import requests
from pprint import pprint
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import re
cookies = {
    'ccsid': '027-9406763-1350835',
    '__qca': 'P0-867993267-1671835417644',
    'p': 'BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa',
    'fbl': 'true',
    'likely_has_account': 'true',
    'srb_8': '0_ar',
    'locale': 'en',
    'blocking_sign_in_interstitial': 'true',
    'u': 'iOMHjnqA6e5wfufL-2lEupl4JJzcKeuyLcy4n61STVNcSHNl',
    '_session_id2': '017c8ce74032994be3d0cb0861f27d69',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=027-9406763-1350835; __qca=P0-867993267-1671835417644; p=BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa; fbl=true; likely_has_account=true; srb_8=0_ar; locale=en; blocking_sign_in_interstitial=true; u=iOMHjnqA6e5wfufL-2lEupl4JJzcKeuyLcy4n61STVNcSHNl; _session_id2=017c8ce74032994be3d0cb0861f27d69',
    'If-None-Match': 'W/"0e6733c40f5c1e1d30624f217a4861d8"',
    'Referer': 'https://www.goodreads.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'ref': 'nav_comm_people',
}

response = requests.get('https://www.goodreads.com/user/best_reviewers', params=params, cookies=cookies, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [259]:
tab = soup.find("table",{"class":"tableList"})

In [260]:
users = []
trs = tab.find_all('tr')
for t in trs: 
    s = t.find_all('td')[1].find('a',href = True)['href']
    user_id = re.findall(r'\d+',s)[0]
    users.append(user_id)

In [262]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from tqdm import tqdm
base_site = "https://www.goodreads.com/list/show/1.Best_Books_Ever?"

In [ ]:
user_ID_1 = []
book_description_1 = []
book_author_1 = []
book_title_1 = []
book_rating_1 = []
book_pages_1 = []
num_of_rating_1 = []
num_of_review_1 = []
genre_1 = []
publish_1 = []
language_1 = []
award_1 = []
isbn_1 = []

for user in users:
    links=[]
    for num in [1,2,3]:
        response1 = requests.get("https://www.goodreads.com/review/list/"+user+"?page="+str(num)
                                 , params=params, cookies=cookies, headers=headers)
        soup = BeautifulSoup(response1.content, 'html.parser')

        tab_book = soup.find("table",{"class":"table stacked"})

        tab_book_1 = tab_book.find_all('tr')

        
        for t in tab_book_1: 
            s = t.find('a',href = True)['href']
            links.append(s)
        
    full_url = [urljoin(base_site, relativeurl) for relativeurl in links]

    book_url = [url for url in full_url if "/book/show" in url]
    
    while 'https://www.goodreads.com/book/show/63337133-rouge' in book_url: 
        book_url.remove('https://www.goodreads.com/book/show/63337133-rouge')
    while 'https://www.goodreads.com/book/show/63141623-house-woman' in book_url: 
        book_url.remove('https://www.goodreads.com/book/show/63141623-house-woman')
    
    user_ID = []
    book_description = []
    book_author = []
    book_title = []
    book_rating = []
    book_pages = []
    num_of_rating = []
    num_of_review = []
    genre = []
    publish = []
    language = []
    award = []
    isbn = []

    import re
    import json

    faulty=[]

    for url in tqdm(book_url):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
        note_resp = requests.get(url, headers=headers,params=params, cookies=cookies)

        #if note_resp.status_code == 200:
         #   print("URL{}: {}".format(i+1, url))

        #else:
         #   print('Status code{}: Skipping URL #{}: {}'.format(note_resp.status_code, i+1, url))
          #  i = i+1
           # continue

        try:
            #get HTML from url page
            note_html = note_resp.content

            #create beautifulsoup object for url page
            note_soup = BeautifulSoup(note_html, 'html.parser')

            #Extract Author particulars


            pages_divs = note_soup.find_all("p", {"data-testid": "pagesFormat"})
            pages_text = pages_divs[0].text


            num_of_rating_divs = note_soup.find_all("span", {"data-testid": "ratingsCount"})
            num_of_rating_text = num_of_rating_divs[0].text


            num_of_review_divs = note_soup.find_all("span", {"data-testid": "reviewsCount"})
            num_of_review_text = num_of_review_divs[0].text


            genre_divs = note_soup.find_all("a", {"class": "Button Button--tag-inline Button--small"})
            genre_text = genre_divs[1].text


            publish_divs = note_soup.find_all("p", {"data-testid":"publicationInfo"})
            publish_text = publish_divs[0].text

            author_divs = note_soup.find_all("span",{"class":"ContributorLink__name"})
            author_text = author_divs[0].text



            title_divs = note_soup.find_all("h1", {"class": "Text Text__title1"})
            title_text = title_divs[0].text


            rating_divs = note_soup.find_all("div", {"class": "RatingStatistics__rating"})
            rating_text = rating_divs[0].text


            json_str = note_soup.find_all("script",{'type':'application/ld+json'})[0].string
            data = json.loads(json_str)


            if 'awards' in data.keys():
                award.append(1)
            else :
                award.append(0)

            if 'isbn' in data.keys():
                isbn.append(data.get("isbn"))
            else :
                isbn.append('NA')


            description_divs = note_soup.find_all("span", {"class": "Formatted"})
            try:
                description_text = description_divs[0].text
            except IndexError:
                description_text = "Nil"


            book_author.append(author_text)
            book_description.append(description_text)
            book_title.append(title_text)
            book_rating.append(float(rating_text))
            book_pages.append(float(re.search(r'\d+', pages_text.replace(',', '')).group()))
            num_of_rating.append(float(re.search(r'\d+', num_of_rating_text.replace(',', '')).group()))
            num_of_review.append(float(re.search(r'\d+', num_of_review_text.replace(',', '')).group()))
            genre.append(genre_text)
            publish.append(publish_text.replace("First published ", ""))
            language.append(data.get("inLanguage"))
            user_ID.append(user)

        except :
            faulty.append(url)


    book_description = book_description[:len(genre)]
    book_author = book_author[:len(genre)]
    book_title = book_title[:len(genre)]
    book_rating = book_rating[:len(genre)]
    book_pages = book_pages[:len(genre)]
    num_of_rating = num_of_rating[:len(genre)]
    num_of_review = num_of_review[:len(genre)]
    genre = genre[:len(genre)]
    publish = publish[:len(genre)]
    language = language[:len(genre)]
    award = award[:len(genre)]
    isbn = isbn[:len(genre)]
    user_ID = user_ID[:len(genre)]
    
    user_ID_1.extend(user_ID)
    book_description_1.extend(book_description)
    book_author_1.extend(book_author)
    book_title_1.extend(book_title)
    book_rating_1.extend(book_rating)
    book_pages_1.extend(book_pages)
    num_of_rating_1.extend(num_of_rating)
    num_of_review_1.extend(num_of_review)
    genre_1.extend(genre)
    publish_1.extend(publish)
    language_1.extend(language)
    award_1.extend(award)
    isbn_1.extend(isbn)

    

In [268]:
final=pd.DataFrame({'Description':book_description_1,
    'Author':book_author_1,
    'Title':book_title_1,
    'Rating':book_rating_1,
    'Pages':book_pages_1,
    'Num of Ratings':num_of_rating_1,
    'Num of Review':num_of_review_1,
    'Genre':genre_1,
    'Publish Date':publish_1,
    'Language':language_1,
    'Award':award_1,
    'ISBN':isbn_1,
    'USERID':user_ID_1})